# Example workflow for postprocessing images

## Import packages

In [3]:
import numpy as np
from math import log2,ceil
import cv2 as cv

## Compare high-speed (10bitADC) with low-speed (14bitADC) image with respect to the number of pixel levels.

In [25]:
print('10bitADC:')
img = cv.imread('./tests/image20190905012252.png', cv.IMREAD_ANYCOLOR | cv.IMREAD_ANYDEPTH )
print('# of Channels: %d / Datatype: %s' % (img.shape[2], str(img.dtype)))
unique, counts = np.unique(img.flatten(), return_counts=True)
counts = len(dict(zip(unique, counts)))
bits = ceil(log2(counts))
print('# of distinct levels: %d (2^%d=%d)' % ( counts, bits, 2**bits))
print('14bitADC:')
img = cv.imread('./tests/image20190915222325.png', cv.IMREAD_ANYCOLOR | cv.IMREAD_ANYDEPTH )
print('# of Channels: %d / Datatype: %s' % (img.shape[2], str(img.dtype)))
unique, counts = np.unique(img.flatten(), return_counts=True)
counts = len(dict(zip(unique, counts)))
bits = ceil(log2(counts))
print('# of distinct levels: %d (2^%d=%d)' % ( counts, bits, 2**bits))

10bitADC:
# of Channels: 3 / Datatype: uint16
# of distinct levels: 32173 (2^15=32768)
14bitADC:
# of Channels: 3 / Datatype: uint16
# of distinct levels: 65449 (2^16=65536)


# Insert new lines here

In [ ]:
import skimage
from skimage.morphology import watershed, disk
from skimage.filters import threshold_otsu, try_all_threshold, rank
from skimage.color import rgb2gray
from skimage.io import imread
from skimage.exposure import rescale_intensity

from scipy import ndimage as ndi

import matplotlib
import matplotlib.pyplot as plt
import sep

Read in test files for testing the algorithm

In [ ]:
#img = rgb2gray(imread('./tests/image20190403215022.png'))
#img = rgb2gray(imread('./tests/image20190905012252.png'))
#example image with 14bitADC (low speed mode) setting
img = rgb2gray(imread('./images/20190915/image20190915221424.png'))
matplotlib.rcParams['font.size'] = 9
img = np.flip(img, 0)

In [ ]:
# show the image
m, s = np.mean(img), np.std(img)
fig, ax = plt.subplots(figsize=(20, 20))
im = ax.imshow(img, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
fig.colorbar(im, ax=ax);

# measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(img)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.figure(figsize = (15,15))
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.figure(figsize = (15,15))
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
# subtract the background
img_sub = img - bkg
# show the image with background subtracted
plt.figure(figsize = (15,15))
plt.imshow(img_sub, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

First do contrast enhancement by stretching the histogram

In [ ]:
p2, p98 = np.percentile(img, (2, 98))
imgo = rescale_intensity(img, in_range=(p2, p98))

Do segmentation on grayscale image and denoise it

In [ ]:
gray = skimage.img_as_ubyte(skimage.color.rgb2gray(imgo))
denoised = rank.median(gray,disk(2))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(gray,cmap='gray')
ax.set_axis_off()
plt.tight_layout()

Plot histogram of image

In [ ]:
plt.hist(gray.ravel(),bins=256,range=(1,254));

Mark sections for watershed algorithm:
find continuous region (low gradient - where less than 10 for this image) --> markers
disk(10) is used here to get a more smooth image

In [ ]:
markers = rank.gradient(denoised, disk(10)) < 15
markers = ndi.label(markers)[0]
fig, ax = plt.subplots(figsize=(20, 20))
plt.imshow(markers, cmap=plt.cm.nipy_spectral, interpolation='nearest')

Show elevation map of image

In [ ]:
from skimage.filters import sobel
elevation_map = sobel(gray)

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(elevation_map, cmap=plt.cm.gray, interpolation='nearest')
ax.set_title('elevation map')
ax.axis('off');

Do watershed segmentation

In [ ]:
labels = watershed(elevation_map, markers)

In [ ]:
plt.subplots(figsize=(15, 15))
plt.imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest', alpha=.7)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16, 10), sharex=True, sharey=True,
                       subplot_kw={'adjustable': 'box-forced'})
ax0, ax1, ax2, ax3 = ax.ravel()
plt.tight_layout()

ax0.imshow(img, cmap=plt.cm.gray)
ax0.set_title('Original')
ax0.axis('off')

ax2.imshow(try_all, cmap=plt.cm.gray)
ax2.set_title('Original >= Local Otsu' % threshold_global_otsu)
ax2.axis('off')

ax3.imshow(global_otsu, cmap=plt.cm.gray)
ax3.set_title('Global Otsu (threshold = %d)' % threshold_global_otsu)
ax3.axis('off')

plt.tight_layout()
plt.show()